In [3]:
# Imports
import json, os, re, shutil, sys, time
import seaborn as sns
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML
from collections import defaultdict

# NLTK for NLP utils and corpora
import nltk
from nltk.corpus import treebank
from nltk.text import Text
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

# Import spacy
import spacy

import pickle

import sklearn
from sklearn.model_selection import train_test_split

# NumPy, Pandas and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf

from numpy.random import seed
from pandas import read_csv, DataFrame
from sklearn.preprocessing import minmax_scale

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.embeddings import Embedding
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Activation, Dropout, Conv1D, MaxPooling1D, Bidirectional, Flatten, TimeDistributed

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from datetime import datetime


Using TensorFlow backend.


## AEG Long Essay Sentence Level Classification

In [6]:
# Read the data
aeg_long = pd.read_csv("../data-DNC/AEG/training_set_rel3.tsv",sep='\t',encoding = "latin1")
aeg_long.shape

(12976, 28)

In [5]:
# Split the data into train and test. We need to do this first to ensure that when we split to 
# sentence level, we have sentences of a given essay in either training or test but not on both.

tr_essay,ts_essay,tr_domain_score,ts_domain_score,tr_essay_id,ts_essay_id,tr_essay_set,ts_essay_set=train_test_split(
    np.asarray(aeg_long.essay),np.asarray(aeg_long.domain1_score),
    np.asarray(aeg_long.essay_id),np.asarray(aeg_long.essay_set),test_size=0.2)

In [6]:
# Check the shapes of training and test datasets
print(tr_essay.shape)
print(ts_essay.shape)

(10380,)
(2596,)


In [10]:
# Create train and test data frames with relevant fields

x_train_df = pd.DataFrame([tr_essay_id,tr_essay_set,tr_essay,tr_domain_score.astype(np.double)]).transpose()
x_train_df.columns = ['essay_id','essay_set','essay','domain1_score']
x_test_df = pd.DataFrame([ts_essay_id,ts_essay_set,ts_essay,ts_domain_score.astype(np.double)]).transpose()
x_test_df.columns = ['essay_id','essay_set','essay','domain1_score']
#print(x_train_df.head(5))
#print(x_test_df.head(5))

#### Split essay into sentences

In [7]:
# We will use spacy to split the essay into sentences. 
# Load spacy large english module
nlp = spacy.load('en_core_web_lg')

In [14]:
# Define a function to split essay into sentences
def create_sentences_df(df):
    """ Function to split essay data into individual sentences. Returns a dataframe"""
    start=datetime.now()
    aeg_long_sentence = pd.DataFrame(columns=['essay_id','essay_set','sentence','domain1_score'])
    for i in range(len(df)):
        if i%1000 == 0:
            print("At iteration :",i)
            print("Duration: ",datetime.now()-start)
        sentence = nlp(df.essay[i])
        for s in sentence.sents:
            aeg_long_sentence = aeg_long_sentence.append({'essay_id' : df.essay_id[i],
                                                          'essay_set' : df.essay_set[i],'sentence' : s.text, 
                                                          'domain1_score' : df.domain1_score[i]},
                                                         ignore_index=True)
    return aeg_long_sentence

In [ ]:
# We also need to convert the spacy format text into regular text
def spacy_to_text(essay):
    return essay[2].text

In [15]:
# Split train data into sentences
x_train_sentence_df = create_sentences_df(x_train_df)

At iteration : 0
Duration:  0:00:00.001094
At iteration : 1000
Duration:  0:01:13.910230
At iteration : 2000
Duration:  0:02:41.471731
At iteration : 3000
Duration:  0:04:57.690647
At iteration : 4000
Duration:  0:07:51.580886
At iteration : 5000
Duration:  0:11:39.480142
At iteration : 6000
Duration:  0:16:11.116163
At iteration : 7000
Duration:  0:21:46.361166
At iteration : 8000
Duration:  0:27:57.529110
At iteration : 9000
Duration:  0:34:42.670452
At iteration : 10000
Duration:  0:42:12.653912


In [17]:
# Split test data into sentences
x_test_sentence_df = create_sentences_df(x_test_df)

At iteration : 0
Duration:  0:00:00.001098
At iteration : 1000
Duration:  0:01:15.384433
At iteration : 2000
Duration:  0:02:53.557499


#### Normalize the scores

In [19]:
# Each essay set has a different scoring range. We need to normalize the scores to a standard scale for training.
def normalize_score(essay):
    """ Normalizes the domain score based on percentage"""
    score = 0
    score = float(essay[3])
    essay_set = essay[1]
    if essay_set == 1:
        div = 12
    elif essay_set == 2:
        div = 5
    elif essay_set == 3:
        div = 3
    elif essay_set == 4:
        div = 3
    elif essay_set == 5:
        div = 4
    elif essay_set == 6:
        div = 4
    elif essay_set == 7:
        div = 25
    elif essay_set == 8:
        div = 50
    return score/div

In [20]:
x_train_sentence_df['Norm_Score'] = x_train_sentence_df.apply(normalize_score,axis=1)
x_test_sentence_df['Norm_Score'] = x_test_sentence_df.apply(normalize_score,axis=1)

In [25]:
x_train_sentence_df.head(5)

,essay_id,essay_set,sentence,domain1_score,Norm_Score
0,4075,2,Did your child ever bring home a book?,3.0,0.6
1,4075,2,"A piece of music, movie or magazine?",3.0,0.6
2,4075,2,Did you ever stop to think that the piece of i...,3.0,0.6
3,4075,2,There are points in time where there isn't an ...,3.0,0.6
4,4075,2,The driving question is do you want to remove ...,3.0,0.6


In [61]:
# Store this dataset in pickle format so that we don't have to redo the above steps.
x_train_sentence_df.to_pickle("./x_train_sentence_df.pkl")
x_test_sentence_df.to_pickle("./x_test_sentence_df.pkl")

In [11]:
# Load the dataset from pickle. 
x_train_sentence_df = pd.read_pickle("/home/pkurapati/x_train_sentence_df.pkl")
x_test_sentence_df = pd.read_pickle("/home/pkurapati/x_test_sentence_df.pkl")

In [12]:
# Create train and test text and labels

x_train = x_train_sentence_df['sentence'].values
y_train = x_train_sentence_df['Norm_Score'].values
x_test = x_test_sentence_df['sentence'].values
y_test = x_test_sentence_df['Norm_Score'].values
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(147647,)
(147647,)
(36254,)
(36254,)


In [13]:
vocabulary_size = 50000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(x_train)
train_seq = tokenizer.texts_to_sequences(x_train)
train_data = pad_sequences(train_seq)

In [14]:
max_len_class = train_data.shape[1]
max_words_class = vocabulary_size

In [15]:
test_seq = tokenizer.texts_to_sequences(x_test)
test_data = pad_sequences(test_seq, maxlen=max_len_class)
test_data.shape

(36254, 175)

In [16]:
embeddings_index = dict()
f = open('/home/pkurapati/W266-NLP-Project/data-DNC/glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1917494 word vectors.


In [17]:
embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [18]:
def RMSE(actual, predict):
    diff = actual - predict
    diff = sum(diff**2) / len(actual)
    return np.sqrt(diff)

In [19]:
#Cohen Kappa score as defined by the kaggle challenge/wikipedia
def CohenKappa(actual, predict):
    CohenDF = pd.DataFrame([actual.astype(np.double).round(), np.around(predict.astype(np.double))]).transpose()
    count = len(CohenDF)
    CohenDF.columns = ['actual','predict']
    correct = len(CohenDF[CohenDF.actual==CohenDF.predict])
    acc = correct / count
    pe = 0
    for value in CohenDF.actual.unique():
        pe += len(CohenDF[CohenDF.actual == value]) * len(CohenDF[CohenDF.predict == value])
    pe = pe / np.square(count)
    print("Count: ",count)
    print("pe:" ,pe)
    return(1 - (1-acc)/(1-pe))

In [35]:
#model definitions

def FF_NN():
    """ Simple feed forward NN"""
    model_ff = Sequential()
    model_ff.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_ff.add(tf.keras.layers.Flatten())
    model_ff.add(tf.keras.layers.Dense(50,activation='tanh'))
    model_ff.add(tf.keras.layers.Dropout(0.1))
    model_ff.add(tf.keras.layers.Dense(1,activation='sigmoid'))
    model_ff.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_ff

def GRU():
    """ Gated Recurrent Unit"""
    model_gru = Sequential()
    model_gru.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_gru.add(tf.keras.layers.GRU(32,activation='tanh'))
    model_gru.add(tf.keras.layers.Dropout(0.1))
    model_gru.add(tf.keras.layers.Dense(1,name='out_layer'))
    model_gru.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_rnn

def CNN_FF():
    """ CNN with Feed Forward NN """
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
    model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=4))
    model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    return model_conv

def CNN_lstm():
    """ CNN with single layer LSTM & Feed Forward NN"""
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
    model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=4))
    model_conv.add(tf.keras.layers.LSTM(100))
    model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    return model_conv

def stack_lstm():
    """ Three layered stacked LSTM."""
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_conv.add(tf.keras.layers.LSTM(32,return_sequences=True))
    model_conv.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model_conv.add(tf.keras.layers.Dropout(0.2))
    model_conv.add(tf.keras.layers.LSTM(32))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    return model_conv

def stateful_stacked_lstm():
    # In stateful, total samples needs to be divisible by batch size
    # we have 147026 samples, so selecting 6683 (6683*22=147026)
    # The test sample need to be a multiple of 6683 as well
    batch_size=2
    model_conv = Sequential()
    # In stateful, we have to pass batch_input_shape to the first layer
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], 
                                             trainable=False,batch_input_shape=(batch_size,max_len_class)))
    model_conv.add(tf.keras.layers.LSTM(32,stateful=True,return_sequences=True))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.LSTM(32))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    #sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    #model_conv.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_conv


#### Common Functions

In [27]:
# Define function to get the multiplication factor. We can get it back from the essay set, but
# it is better to do it from the score, because there are scores with value 0, and its MF should be 0

def find_mult_factor(x):
    """ Function to find the multiplication factor for denormalizing"""
    if x[1] == 0:
        return 0
    else:
        return np.around(x[0]/x[1])

In [28]:
def denormalize(x):
    """ Function to Denormalize the score"""
    return np.around(x[2] * x[3])

In [29]:
def find_max_min_mean_score(df):
    """ Function to find the max, min and rounded mean of sentence scores"""
    new_df = pd.DataFrame(columns=['essay_id','essay_set','Orig_Score','Max_Score','Min_Score','Mean_Score'])
    essay_ids = np.unique(df.essay_id)
    for e_id in essay_ids:
        df_temp = df[df.essay_id == e_id]
        max_score = np.max(df_temp.pred_score)
        min_score = np.min(df_temp.pred_score)
        # we need to round the mean so that kappa score doesnt complain
        mean_score = np.around(np.mean(df_temp.pred_score))
        new_df = new_df.append({'essay_id':e_id,'essay_set':int(np.unique(df_temp.essay_set)),
                                'Orig_Score':int(np.unique(df_temp.orig_score)),'Max_Score':max_score,
                                'Min_Score':min_score,'Mean_Score':mean_score},ignore_index=True)
    return new_df

In [30]:
# To calculate the per essay set scores

def essay_set_metrics(df):
    """ Calculate per essay set metrics"""
    set_df = pd.DataFrame(columns=['essay_set','RMSE','Kappa','Accuracy'])
    e_sets = np.unique(df.essay_set)
    for e_s in e_sets:
        df_s = df[df.essay_set == e_s]
        original_score = df_s.Orig_Score.values.astype(int)
        predicted_score = df_s.Mean_Score.values.astype(int)
        rmse = RMSE(original_score,predicted_score)
        kappa = cohen_kappa_score(original_score,predicted_score)
        accuracy = accuracy_score(original_score,predicted_score)
        set_df = set_df.append({'essay_set':e_s,'RMSE':rmse,'Kappa':kappa,'Accuracy':accuracy},
                              ignore_index=True)
    return set_df

In [ ]:
def sentence_count(df):
    """ Returns the number of sentences in an essay """
    essay_count = df.groupby('essay_id').count()
    essay_count = essay_count.drop(['sentence','domain1_score','Norm_Score'],axis=1)
    essay_count.columns = ['Number_of_Sentences']
    return essay_count

#### CNN_LSTM

In [31]:
# Train & Predict
estimator = KerasRegressor(build_fn=CNN_lstm, epochs=20, batch_size=500)
estimator.fit(train_data, y_train)
prediction_cnn_glove=estimator.predict(test_data)
# Metric before denormalizing. This is not very useful as the scale per essay set is different
rmse_val = RMSE(y_test,prediction_cnn_glove)
print("RMSE before denormalizing: ",rmse_val)

Epoch 1/20
147647/147647 [==============================] - 40s 269us/step - loss: 0.0383 - acc: 0.0980
Epoch 2/20
147647/147647 [==============================] - 26s 176us/step - loss: 0.0288 - acc: 0.1003
Epoch 3/20
147647/147647 [==============================] - 26s 178us/step - loss: 0.0280 - acc: 0.1007
Epoch 4/20
147647/147647 [==============================] - 26s 178us/step - loss: 0.0273 - acc: 0.1012
Epoch 5/20
147647/147647 [==============================] - 26s 177us/step - loss: 0.0265 - acc: 0.1014
Epoch 6/20
147647/147647 [==============================] - 27s 180us/step - loss: 0.0258 - acc: 0.1018
Epoch 7/20
147647/147647 [==============================] - 26s 178us/step - loss: 0.0254 - acc: 0.1021
Epoch 8/20
147647/147647 [==============================] - 27s 181us/step - loss: 0.0246 - acc: 0.1026
Epoch 9/20
147647/147647 [==============================] - 27s 180us/step - loss: 0.0241 - acc: 0.1029
Epoch 10/20
147647/147647 [==============================] - 27s

In [32]:
# Denormalize

# Construct pandas data frame of scores of each sentences
cnn_df = pd.DataFrame([x_test_sentence_df['domain1_score'].astype(np.double), y_test.astype(np.double),
                      prediction_cnn_glove.astype(np.double)]).transpose()
cnn_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
# Find Multiplication factor using the function we defined before
cnn_df['Mult_Factor'] = cnn_df.apply(find_mult_factor,axis=1)
cnn_df['Denorm_Pred_Score'] = cnn_df.apply(denormalize,axis=1)

# Extract the scores of each sentences
orig_score = cnn_df.Orig_Score.as_matrix(columns=None)
orig_score = orig_score.astype(np.int)
pred_score = cnn_df.Denorm_Pred_Score.as_matrix(columns=None)
pred_score = pred_score.astype(np.int)
rmse_cnn = RMSE(orig_score,pred_score)

# Provide the metrics at sentence level
print("RMSE: ",rmse_cnn)
cohen_kappa = cohen_kappa_score(orig_score,pred_score)
print("Cohen Kappa: ",cohen_kappa)
accuracy = accuracy_score(orig_score,pred_score)
print("Accuracy: ",accuracy)

RMSE:  2.6965874406799935
Cohen Kappa:  0.26002881898405206
Accuracy:  0.32608815578970596


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [33]:
# We will combine the sentences back to essay in this section
result_df = pd.DataFrame([x_test_sentence_df.essay_id.values,x_test_sentence_df.essay_set.values,
                         x_test_sentence_df.sentence.values,cnn_df.Orig_Score.values,
                          cnn_df.Denorm_Pred_Score.values]).transpose()
result_df.columns = ['essay_id','essay_set','sentence','orig_score','pred_score']
cnn_test_df = find_max_min_mean_score(result_df)

#Convert the dataframe parameters into integers as they are returned as floats
cnn_test_df.essay_id = cnn_test_df.essay_id.astype(int)
cnn_test_df.essay_set = cnn_test_df.essay_set.astype(int)
cnn_test_df.Orig_Score = cnn_test_df.Orig_Score.astype(int)
cnn_test_df.Max_Score = cnn_test_df.Max_Score.astype(int)
cnn_test_df.Min_Score = cnn_test_df.Min_Score.astype(int)
cnn_test_df.Mean_Score = cnn_test_df.Mean_Score.astype(int)

# Extract the scores
orig_score = cnn_test_df.Orig_Score.values
max_pred_score = cnn_test_df.Max_Score.values
min_pred_score = cnn_test_df.Min_Score.values
mean_pred_score = cnn_test_df.Mean_Score.values

# Compare the metrics if Max score of all sentences are taken
rmse_max_cnn = RMSE(orig_score,max_pred_score)
cohen_kappa_max_cnn = cohen_kappa_score(orig_score,max_pred_score)
accuracy_max_cnn = accuracy_score(orig_score,max_pred_score)

# Compare the metrics if Min score of all sentences are taken
rmse_min_cnn = RMSE(orig_score,min_pred_score)
cohen_kappa_min_cnn = cohen_kappa_score(orig_score,min_pred_score)
accuracy_min_cnn = accuracy_score(orig_score,min_pred_score)

# Compare the metrics if Rounded Mean score of all sentences are taken
rmse_mean_cnn = RMSE(orig_score,mean_pred_score)
cohen_kappa_mean_cnn = cohen_kappa_score(orig_score,mean_pred_score)
accuracy_mean_cnn = accuracy_score(orig_score,mean_pred_score)

print("### CNN_LSTM Results : MAX Score of all Sentences ###")
print("RMSE: ",rmse_max_cnn)
print("Cohen Kappa: ",cohen_kappa_max_cnn)
print("Accuracy: ",accuracy_max_cnn)

print("### CNN_LSTM Results : Min Score of all Sentences ###")
print("RMSE: ",rmse_min_cnn)
print("Cohen Kappa: ",cohen_kappa_min_cnn)
print("Accuracy: ",accuracy_min_cnn)

print("### CNN_LSTM Results : Mean Score of all Sentences ###")
print("RMSE: ",rmse_mean_cnn)
print("Cohen Kappa: ",cohen_kappa_mean_cnn)
print("Accuracy: ",accuracy_mean_cnn)

# Provide per-essay-set metrics based on mean score
essay_set_results = essay_set_metrics(cnn_test_df)
essay_set_results

### CNN_LSTM Results : MAX Score of all Sentences ###
RMSE:  3.1392318454285677
Cohen Kappa:  0.17031510015035434
Accuracy:  0.262326656394453
### CNN_LSTM Results : Min Score of all Sentences ###
RMSE:  2.8175107307741944
Cohen Kappa:  0.2586769081038869
Accuracy:  0.35824345146379044
### CNN_LSTM Results : Mean Score of all Sentences ###
RMSE:  2.136970048019511
Cohen Kappa:  0.2707089966704237
Accuracy:  0.3647919876733436


,essay_set,RMSE,Kappa,Accuracy
0,1.0,1.454219,0.042462,0.240437
1,2.0,0.869577,0.072927,0.476712
2,3.0,0.769659,0.081901,0.434018
3,4.0,0.643650,0.408171,0.594286
4,5.0,1.007968,0.067819,0.338667
5,6.0,0.884122,0.072215,0.474394
6,7.0,4.695649,-0.003495,0.066667
7,8.0,4.949394,0.007603,0.048951


#### Stacked LSTM

In [37]:
# Train the model
estimator_lstm = KerasRegressor(build_fn=stack_lstm, epochs=20, batch_size=500)
estimator_lstm.fit(train_data, y_train)

# Predict for test data
prediction_lstm=estimator_lstm.predict(test_data)
rmse_val = RMSE(y_test,prediction_lstm)
# Find the overall RMSE value. This is not very relevant considering different scale for each set.
print("RMSE: ",rmse_val)

# Construct pandas data frame of scores of each sentences
lstm_df = pd.DataFrame([x_test_sentence_df['domain1_score'].astype(np.double), y_test.astype(np.double),
                      prediction_lstm.astype(np.double)]).transpose()
lstm_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
# Find Multiplication factor using the function we defined before
lstm_df['Mult_Factor'] = lstm_df.apply(find_mult_factor,axis=1)
# Find the denormalized predicted score
lstm_df['Denorm_Pred_Score'] = lstm_df.apply(denormalize,axis=1)

# Extract the scores of each sentences
orig_score = lstm_df.Orig_Score.as_matrix(columns=None)
orig_score = orig_score.astype(np.int)
pred_score = lstm_df.Denorm_Pred_Score.as_matrix(columns=None)
pred_score = pred_score.astype(np.int)

# Provide the metrics at sentence level
rmse_lstm = RMSE(orig_score,pred_score)
cohen_kappa = cohen_kappa_score(orig_score,pred_score)
accuracy = accuracy_score(orig_score,pred_score)
print("RMSE Sentence Level: ",rmse_lstm)
print("Kappa Sentence Level: ",cohen_kappa)
print("Accuracy Sentence Level: ",accuracy)

Epoch 1/20
147647/147647 [==============================] - 315s 2ms/step - loss: 0.0462 - acc: 0.0960
Epoch 2/20
147647/147647 [==============================] - 309s 2ms/step - loss: 0.0307 - acc: 0.0999
Epoch 3/20
 61500/147647 [===========>..................] - ETA: 2:59 - loss: 0.0287 - acc: 0.1024

KeyboardInterrupt: 

In [ ]:
# We will combine the sentences back to essay in this section

result_df = pd.DataFrame([x_test_sentence_df.essay_id.values,x_test_sentence_df.essay_set.values,
                         x_test_sentence_df.sentence.values,lstm_df.Orig_Score.values,
                          lstm_df.Denorm_Pred_Score.values]).transpose()
result_df.columns = ['essay_id','essay_set','sentence','orig_score','pred_score']
lstm_test_df = find_max_min_mean_score(result_df)

#Convert the dataframe parameters into integers as they are returned as floats
lstm_test_df.essay_id = lstm_test_df.essay_id.astype(int)
lstm_test_df.essay_set = lstm_test_df.essay_set.astype(int)
lstm_test_df.Orig_Score = lstm_test_df.Orig_Score.astype(int)
lstm_test_df.Max_Score = lstm_test_df.Max_Score.astype(int)
lstm_test_df.Min_Score = lstm_test_df.Min_Score.astype(int)
lstm_test_df.Mean_Score = lstm_test_df.Mean_Score.astype(int)

# Extract the scores
orig_score = lstm_test_df.Orig_Score.values
max_pred_score = lstm_test_df.Max_Score.values
min_pred_score = lstm_test_df.Min_Score.values
mean_pred_score = lstm_test_df.Mean_Score.values

# Compare the metrics if Max score of all sentences are taken
rmse_max_lstm = RMSE(orig_score,max_pred_score)
cohen_kappa_max_lstm = cohen_kappa_score(orig_score,max_pred_score)
accuracy_max_lstm = accuracy_score(orig_score,max_pred_score)

# Compare the metrics if Min score of all sentences are taken
rmse_min_lstm = RMSE(orig_score,min_pred_score)
cohen_kappa_min_lstm = cohen_kappa_score(orig_score,min_pred_score)
accuracy_min_lstm = accuracy_score(orig_score,min_pred_score)

# Compare the metrics if Mean score of all sentences are taken
rmse_mean_lstm = RMSE(orig_score,mean_pred_score)
cohen_kappa_mean_lstm = cohen_kappa_score(orig_score,mean_pred_score)
accuracy_mean_lstm = accuracy_score(orig_score,mean_pred_score)

print("### Stacked LSTM Results : MAX Score of all Sentences ###")
print("RMSE: ",rmse_max_lstm)
print("Cohen Kappa: ",cohen_kappa_max_lstm)
print("Accuracy: ",accuracy_max_lstm)

print("### Stacked LSTM Results : Min Score of all Sentences ###")
print("RMSE: ",rmse_min_lstm)
print("Cohen Kappa: ",cohen_kappa_min_lstm)
print("Accuracy: ",accuracy_min_lstm)

print("### Stacked LSTM Results : Mean Score of all Sentences ###")
print("RMSE: ",rmse_mean_lstm)
print("Cohen Kappa: ",cohen_kappa_mean_lstm)
print("Accuracy: ",accuracy_mean_lstm)

# Provide per-essay-set metrics based on mean score
essay_set_results = essay_set_metrics(lstm_test_df)
essay_set_results

#### RNN GRU

In [133]:
# Train the model
estimator = KerasRegressor(build_fn=CNN_glove_1, epochs=20, batch_size=500)
estimator.fit(train_data, y_train)

# Predict for test data
prediction_rnn=estimator.predict(test_data)
rmse_val = RMSE(y_test,prediction_rnn)
# Find the overall RMSE value. This is not very relevant considering different scale for each set.
print("RMSE: ",rmse_val)

# Construct pandas data frame of scores of each sentences
rnn_df = pd.DataFrame([x_test_sentence_df['domain1_score'].astype(np.double), y_test.astype(np.double),
                      prediction_rnn.astype(np.double)]).transpose()
rnn_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
# Find Multiplication factor using the function we defined before
rnn_df['Mult_Factor'] = rnn_df.apply(find_mult_factor,axis=1)
# Find the denormalized predicted score
rnn_df['Denorm_Pred_Score'] = rnn_df.apply(denormalize,axis=1)

# Extract the scores of each sentences
orig_score = rnn_df.Orig_Score.as_matrix(columns=None)
orig_score = orig_score.astype(np.int)
pred_score = rnn_df.Denorm_Pred_Score.as_matrix(columns=None)
pred_score = pred_score.astype(np.int)

# Provide the metrics at sentence level
rmse_rnn = RMSE(orig_score,pred_score)
cohen_kappa = cohen_kappa_score(orig_score,pred_score)
accuracy = accuracy_score(orig_score,pred_score)
print("RMSE Sentence Level: ",rmse_rnn)
print("Kappa Sentence Level: ",cohen_kappa)
print("Accuracy Sentence Level: ",accuracy)




Epoch 1/20
147647/147647 [==============================] - 229s 2ms/step - loss: 0.1327 - acc: 0.0923
Epoch 2/20
147647/147647 [==============================] - 239s 2ms/step - loss: 0.0310 - acc: 0.0999
Epoch 3/20
147647/147647 [==============================] - 238s 2ms/step - loss: 0.0299 - acc: 0.1001
Epoch 4/20
147647/147647 [==============================] - 240s 2ms/step - loss: 0.0293 - acc: 0.1003
Epoch 5/20
147647/147647 [==============================] - 229s 2ms/step - loss: 0.0287 - acc: 0.1004
Epoch 6/20
147647/147647 [==============================] - 228s 2ms/step - loss: 0.0284 - acc: 0.1006
Epoch 7/20
147647/147647 [==============================] - 228s 2ms/step - loss: 0.0279 - acc: 0.1007
Epoch 8/20
147647/147647 [==============================] - 227s 2ms/step - loss: 0.0276 - acc: 0.1010
Epoch 9/20
147647/147647 [==============================] - 227s 2ms/step - loss: 0.0274 - acc: 0.1010
Epoch 10/20
147647/147647 [==============================] - 232s 2ms/ste

In [134]:
# We will combine the sentences back to essay in this section

result_df = pd.DataFrame([x_test_sentence_df.essay_id.values,x_test_sentence_df.essay_set.values,
                         x_test_sentence_df.sentence.values,rnn_df.Orig_Score.values,
                          rnn_df.Denorm_Pred_Score.values]).transpose()
result_df.columns = ['essay_id','essay_set','sentence','orig_score','pred_score']
rnn_test_df = find_max_min_mean_score(result_df)

#Convert the dataframe parameters into integers as they are returned as floats
rnn_test_df.essay_id = rnn_test_df.essay_id.astype(int)
rnn_test_df.essay_set = rnn_test_df.essay_set.astype(int)
rnn_test_df.Orig_Score = rnn_test_df.Orig_Score.astype(int)
rnn_test_df.Max_Score = rnn_test_df.Max_Score.astype(int)
rnn_test_df.Min_Score = rnn_test_df.Min_Score.astype(int)
rnn_test_df.Mean_Score = rnn_test_df.Mean_Score.astype(int)

# Extract the scores
orig_score = rnn_test_df.Orig_Score.values
max_pred_score = rnn_test_df.Max_Score.values
min_pred_score = rnn_test_df.Min_Score.values
mean_pred_score = rnn_test_df.Mean_Score.values

# Compare the metrics if Max score of all sentences are taken
rmse_max_rnn = RMSE(orig_score,max_pred_score)
cohen_kappa_max_rnn = cohen_kappa_score(orig_score,max_pred_score)
accuracy_max_rnn = accuracy_score(orig_score,max_pred_score)

# Compare the metrics if Min score of all sentences are taken
rmse_min_rnn = RMSE(orig_score,min_pred_score)
cohen_kappa_min_rnn = cohen_kappa_score(orig_score,min_pred_score)
accuracy_min_rnn = accuracy_score(orig_score,min_pred_score)

# Compare the metrics if Mean score of all sentences are taken
rmse_mean_rnn = RMSE(orig_score,mean_pred_score)
cohen_kappa_mean_rnn = cohen_kappa_score(orig_score,mean_pred_score)
accuracy_mean_rnn = accuracy_score(orig_score,mean_pred_score)

print("### RNN GRU Results : MAX Score of all Sentences ###")
print("RMSE: ",rmse_max_rnn)
print("Cohen Kappa: ",cohen_kappa_max_rnn)
print("Accuracy: ",accuracy_max_rnn)

print("### RNN GRU Results : Min Score of all Sentences ###")
print("RMSE: ",rmse_min_rnn)
print("Cohen Kappa: ",cohen_kappa_min_rnn)
print("Accuracy: ",accuracy_min_rnn)

print("### RNN GRU Results : Mean Score of all Sentences ###")
print("RMSE: ",rmse_mean_rnn)
print("Cohen Kappa: ",cohen_kappa_mean_rnn)
print("Accuracy: ",accuracy_mean_rnn)

# Provide per-essay-set metrics based on mean score
essay_set_results = essay_set_metrics(rnn_test_df)
essay_set_results

### RNN GRU Results : MAX Score of all Sentences ###
RMSE:  2.766462143045881
Cohen Kappa:  0.23434395207267844
Accuracy:  0.32704160246533126
### RNN GRU Results : Min Score of all Sentences ###
RMSE:  2.7515229009165276
Cohen Kappa:  0.24823282567822424
Accuracy:  0.3501540832049307
### RNN GRU Results : Mean Score of all Sentences ###
RMSE:  2.05284574550881
Cohen Kappa:  0.29721906803674936
Accuracy:  0.3898305084745763
Count:  366
pe: 0.2415494640031055
Count:  365
pe: 0.43092512666541566
Count:  341
pe: 0.3934262691239325
Count:  350
pe: 0.3190938775510204
Count:  375
pe: 0.29968355555555554
Count:  371
pe: 0.43327932810717734
Count:  285
pe: 0.07799322868574945
Count:  143
pe: 0.05027140691476356


,essay_set,RMSE,Kappa,Accuracy
0,1.0,1.406464,0.059774,0.286885
1,2.0,0.797256,0.167117,0.526027
2,3.0,0.765840,0.047581,0.422287
3,4.0,0.614120,0.458704,0.631429
4,5.0,0.930949,0.116590,0.381333
5,6.0,0.849925,0.110597,0.495957
6,7.0,4.526724,-0.012285,0.066667
7,8.0,4.744522,-0.023480,0.027972


#### Feed Forward NN

In [139]:
# Train the model
estimator_nn = KerasRegressor(build_fn=FF_NN, epochs=20, batch_size=500)
estimator_nn.fit(train_data, y_train)

# Predict for test data
prediction_nn=estimator_nn.predict(test_data)
rmse_val = RMSE(y_test,prediction_nn)
# Find the overall RMSE value. This is not very relevant considering different scale for each set.
print("RMSE: ",rmse_val)

# Construct pandas data frame of scores of each sentences
nn_df = pd.DataFrame([x_test_sentence_df['domain1_score'].astype(np.double), y_test.astype(np.double),
                      prediction_nn.astype(np.double)]).transpose()
nn_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
# Find Multiplication factor using the function we defined before
nn_df['Mult_Factor'] = nn_df.apply(find_mult_factor,axis=1)
# Find the denormalized predicted score
nn_df['Denorm_Pred_Score'] = nn_df.apply(denormalize,axis=1)

# Extract the scores of each sentences
orig_score = nn_df.Orig_Score.as_matrix(columns=None)
orig_score = orig_score.astype(np.int)
pred_score = nn_df.Denorm_Pred_Score.as_matrix(columns=None)
pred_score = pred_score.astype(np.int)

# Provide the metrics at sentence level
rmse_nn = RMSE(orig_score,pred_score)
cohen_kappa = cohen_kappa_score(orig_score,pred_score)
accuracy = accuracy_score(orig_score,pred_score)
print("RMSE Sentence Level: ",rmse_nn)
print("Kappa Sentence Level: ",cohen_kappa)
print("Accuracy Sentence Level: ",accuracy)

RMSE:  0.17686244661930914
RMSE Sentence Level:  2.872749442331444
Kappa Sentence Level:  0.25184272299747656
Accuracy Sentence Level:  0.31877861753185854


In [140]:
# We will combine the sentences back to essay in this section

result_df = pd.DataFrame([x_test_sentence_df.essay_id.values,x_test_sentence_df.essay_set.values,
                         x_test_sentence_df.sentence.values,nn_df.Orig_Score.values,
                          nn_df.Denorm_Pred_Score.values]).transpose()
result_df.columns = ['essay_id','essay_set','sentence','orig_score','pred_score']
nn_test_df = find_max_min_mean_score(result_df)

#Convert the dataframe parameters into integers as they are returned as floats
nn_test_df.essay_id = nn_test_df.essay_id.astype(int)
nn_test_df.essay_set = nn_test_df.essay_set.astype(int)
nn_test_df.Orig_Score = nn_test_df.Orig_Score.astype(int)
nn_test_df.Max_Score = nn_test_df.Max_Score.astype(int)
nn_test_df.Min_Score = nn_test_df.Min_Score.astype(int)
nn_test_df.Mean_Score = nn_test_df.Mean_Score.astype(int)

# Extract the scores
orig_score = nn_test_df.Orig_Score.values
max_pred_score = nn_test_df.Max_Score.values
min_pred_score = nn_test_df.Min_Score.values
mean_pred_score = nn_test_df.Mean_Score.values

# Compare the metrics if Max score of all sentences are taken
rmse_max_nn = RMSE(orig_score,max_pred_score)
cohen_kappa_max_nn = cohen_kappa_score(orig_score,max_pred_score)
accuracy_max_nn = accuracy_score(orig_score,max_pred_score)

# Compare the metrics if Min score of all sentences are taken
rmse_min_nn = RMSE(orig_score,min_pred_score)
cohen_kappa_min_nn = cohen_kappa_score(orig_score,min_pred_score)
accuracy_min_nn = accuracy_score(orig_score,min_pred_score)

# Compare the metrics if Mean score of all sentences are taken
rmse_mean_nn = RMSE(orig_score,mean_pred_score)
cohen_kappa_mean_nn = cohen_kappa_score(orig_score,mean_pred_score)
accuracy_mean_nn = accuracy_score(orig_score,mean_pred_score)

print("### Simple FF NN Results : MAX Score of all Sentences ###")
print("RMSE: ",rmse_max_nn)
print("Cohen Kappa: ",cohen_kappa_max_nn)
print("Accuracy: ",accuracy_max_nn)

print("### Simple FF NN Results : Min Score of all Sentences ###")
print("RMSE: ",rmse_min_nn)
print("Cohen Kappa: ",cohen_kappa_min_nn)
print("Accuracy: ",accuracy_min_nn)

print("### Simple FF NN Results : Mean Score of all Sentences ###")
print("RMSE: ",rmse_mean_nn)
print("Cohen Kappa: ",cohen_kappa_mean_nn)
print("Accuracy: ",accuracy_mean_nn)

# Provide per-essay-set metrics based on mean score
essay_set_results = essay_set_metrics(nn_test_df)
essay_set_results

### Simple FF NN Results : MAX Score of all Sentences ###
RMSE:  3.105241852828997
Cohen Kappa:  0.20269573315671097
Accuracy:  0.29622496147919875
### Simple FF NN Results : Min Score of all Sentences ###
RMSE:  3.3593807669173277
Cohen Kappa:  0.185911899786948
Accuracy:  0.2935285053929122
### Simple FF NN Results : Mean Score of all Sentences ###
RMSE:  2.002213720930788
Cohen Kappa:  0.2981514485881216
Accuracy:  0.39175654853620956


,essay_set,RMSE,Kappa,Accuracy
0,1.0,1.393780,0.081675,0.311475
1,2.0,0.792084,0.193879,0.539726
2,3.0,0.756206,0.074548,0.436950
3,4.0,0.639196,0.411744,0.600000
4,5.0,1.001332,0.067606,0.341333
5,6.0,0.849925,0.094471,0.487871
6,7.0,4.451690,0.018524,0.101754
7,8.0,4.454117,0.013742,0.062937
